In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

import glob
import sys
sys.path.append("../lib/")
import eyegaze as eg

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
input_dir = "../../data/input/"
working_dir = "../../data/working/"
output_dir = "../../data/output/"
savepath = "../../data/working/feature/"

participants = ["P01","P02","P03","P04","P05","P06","P07","P08","P09",
                "P10","P11","P12","P13","P15","P16","P18","P19"]

def v(df, column):      
    return df[column].values

# def align(fx):
#     line_number = 0
#     for f in fx:
#         if f[4] > 550 and f[5] > 2.0:
#             line_number += 1
#         f[2] = 420 + line_number*45
#         f[1]+= 120
#     return fx


def export_scanpath(participant):
    num_paragraph = pd.read_csv(input_dir+"paragraphs.csv")["Paragraphs"].values

    for document, g in enumerate(sorted(glob.glob(input_dir+"/%s/*/" %participant))):
        #print (document, g)
        data = pd.read_csv(g + "tobii_pro_gaze.csv")
        capture = pd.read_csv(g + "capture.csv")["#timestamp"].values
        #screenshots = len(glob.glob1(g,"*.png"))

        paragraphs = range(2, 2 + num_paragraph[document])
        #screenshots = range(2, 1 + screenshots)
        #print (paragraphs)
        
        for paragraph, i in enumerate(paragraphs):
            
            try:
                
                gaze = data[(data["#timestamp"] > capture[i-2]) & (data["#timestamp"] < capture[i-1])]
                gaze = gaze[["#timestamp", "tobii_system_timestamp", "gaze_x", "gaze_y", "pupil_diameter", "left_gaze_x", "left_gaze_y", "left_pupil", "right_gaze_x", "right_gaze_y", "right_pupil"]]
                #gaze.to_csv(working_dir+"gaze/%s" %participant + "_D"+str(document).zfill(2) +"_p"+str(paragraph+1) +".csv", index=False)
                fx = eg.detectFixations(v(gaze, "#timestamp"), v(gaze, "gaze_x"), v(gaze, "gaze_y"))
                #fx.to_csv(working_dir+"feature/P01"+"_d"+str(document).zfill(2)+"_p"+str(paragraph+1)+".csv", index=False)

#                 eg.plotScanPath(
#                                 fx[:, 1], fx[:, 2], fx[:, 3],
#                                 bg_image = g+str(i).zfill(3)+"_back.png",
#                                 save_path=output_dir+"%s" %participant +"_D"+str(document).zfill(2)+"_p"+str(paragraph+1)+".png")
            except:
                print("Error in: ", participant, document, paragraph)

In [21]:
#for p in participants:
    #export_scanpath(p)

In [51]:
#gaze feature extraction
gaze_dir = working_dir + "gaze/"
for g in (sorted(glob.glob(gaze_dir + "*"))):
    
    gaze_data = pd.read_csv(g)
    features = eg.detectFixations(v(gaze_data, "#timestamp"), v(gaze_data, "gaze_x"), v(gaze_data, "gaze_y"))
    
    try:
        pd.DataFrame(features).to_csv(working_dir + "feature/" + str(g[24:34]) + '.csv', header=['#timestamp','fixation_x','fixation_y','fixation_duration',
                                                                                             'saccade_length','saccade_angle','saccade_velocity'])
    except:
        print ("Error in : ", str(g[24:34]))

Error in :  P05_D02_p3


In [69]:
feature_dir = working_dir + "feature/"
for g in (sorted(glob.glob(feature_dir + "*"))):
    
    try:
        feature_data = pd.read_csv(g)
        feature_data.insert(loc=0, column='participant', value=g[27:30])
        feature_data.insert(loc=1, column='document', value=g[31:34])
        feature_data.insert(loc=2, column='paragraph', value=g[35:37])

        feature_data.to_csv(working_dir + "ft/"+ str(g[27:37]) + '.csv', index=True)
        
    except:
        print ("Error in : ", str(g[27:37]))    

Error in :  P05_D02_p3


In [41]:
ft_dir = working_dir + "ft/"
df_list = [pd.read_csv(feature_list, index_col=0) for feature_list in sorted(glob.glob(ft_dir + "*"))]
feat_comb = pd.concat(df_list, ignore_index=True, sort=False)
feat_comb.drop("Unnamed: 0.1",axis=1)
feat_comb.to_csv(working_dir + "all_data.csv")